# **SemEval-2025 Task 6: A Multi-Architecture Approach for Corporate Environmental Promise Verification - The Base Model (Model 1)**

Model 1 Notebook Author:

- Nawar Turk - https://www.linkedin.com/in/nawart/

Paper Authors:
- Nawar Turk *
- Eeham Khan *
- Leila Kosseim

*Equal contribution



# 1. Introduction


## 1.1. Overview


This notebook addresses the SemEval-2025 Task 6 (PromiseEval) challenge, which focuses on verifying promises in corporate ESG (Environmental, Social, and Governance) reports. The competition consists of four subtasks: promise identification, supporting evidence assessment, clarity evaluation, and verification timing.

We have developed three distinct model architectures of increasing complexity:

* Model 1 (This notebook): Base ESG-BERT with task-specific classification heads
* Model 2: Enhanced ESG-BERT with linguistic features
* Model 3: Combined multi-objective approach with attention mechanisms

## 1.2. The Base Model (Model 1)

Our base model employs ESG-BERT, a domain-specific BERT variant pre-trained on environmental, social, and governance texts. For each subtask, we use the same underlying ESG-BERT architecture but add separate classification heads trained independently:

* Task 1 (Promise Identification): Binary classification (Yes/No)
* Task 2 (Supporting Evidence): Binary classification (Yes/No)
* Task 3 (Clarity Evaluation): Multi-class classification (Clear/Not Clear/Misleading/N/A)
* Task 4 (Verification Timeline): Multi-class classification (Less than 2 years/2 to 5 years/More than 5 years/Already/N/A)

We implement partial fine-tuning by freezing the lower layers of the transformer and only training the top two transformer layers along with the task-specific classification heads. This approach balances leveraging the pre-trained ESG knowledge while adapting to the specific nuances of promise verification.

## 1.3. Links

* Task Website: https://sites.google.com/view/promiseeval/promiseeval




* Competition Page: https://www.kaggle.com/competitions/sem-eval-2025-promise-eval-english/overview

* Our Repository and Paper: https://github.com/CLaC-Lab/SemEval-2025-Task6

* This Notebook: https://colab.research.google.com/drive/1qlOs2B7PWvADnD3TaIluonRC5XqfmJbG?usp=sharing

#  2. Environment Setup & Data Loading


## 2.1. Install dependencies

In [ ]:
%%capture
!pip install --upgrade pip

# Pin all your working versions
!pip install \
    torch==2.6.0+cu124 \
    numpy==2.0.2 \
    pandas==2.2.2 \
    scikit-learn==1.6.1 \
    datasets==3.5.1 \
    optuna==4.3.0 \
    flair==0.15.1 \
    spacy==3.8.5 \
    transformers==4.49.0

# Download the spaCy English model that matches spacy v3.8.5
!python -m spacy download en_core_web_sm

## 2.2 Import libraries

In [ ]:
import json
import torch
import numpy as np
from sklearn.model_selection import StratifiedKFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
import optuna
import os
import random
import pandas as pd

## 2.3. Define Paths

In [ ]:
train_data_path = 'PromiseEval_Trainset_English.json'
test_data_path = 'test_data_unlabelled.json'

#output folders
task1_output_path = './promise_task1_processed_Base-Model'
task2_output_path = './promise_task2_processed_Base-Model'
task3_output_path = './promise_task3_processed_Base-Model'
task4_output_path = './promise_task4_processed_Base-Model'

os.makedirs(task1_output_path, exist_ok=True)
os.makedirs(task2_output_path, exist_ok=True)
os.makedirs(task3_output_path, exist_ok=True)
os.makedirs(task4_output_path, exist_ok=True)

## 2.4. Training & Testing Datasets



### 2.4.1. Getting the Data





Contact the Task organizers of SemEval 2025 PromiseEval to obtain the training data: https://groups.google.com/g/promiseeval

- Train data: "PromiseEval_Trainset_English.json"
- Test data: "english_submission_file.json"

### 2.4.2. Restructuring Test Data



We need to restructure the SemEval 2025 PromiseEval test data to match both our model's input format and the competition submission requirements.

In [ ]:
# Load the original test file
with open("english_submission_file.json", "r") as file:
    original_data = json.load(file)

# Create new structure
restructured_data = {}

# Determine the total number of items (assuming all fields have the same count)
total_items = len(original_data["data"])

# For each index, create a corresponding entry
for idx in range(total_items):
    idx_str = str(idx)
    restructured_data[idx_str] = {
        "ID": original_data["ID"].get(idx_str, ""),
        "URL": original_data["URL"].get(idx_str, ""),
        "page_number": original_data["page_number"].get(idx_str, ""),
        "data": original_data["data"].get(idx_str, ""),
        "promise_status": "",
        "verification_timeline":"",
        "evidence_status":"",
        "evidence_quality":""
    }

# Save the restructured data
with open("english_submission_file_restructured_unlabeled.json", "w") as file:
    json.dump(restructured_data, file, indent=2)

print(f"Restructured data saved with {len(restructured_data)} items.")

Restructured data saved with 400 items.


# 3. The Four Tasks, Train & Predict

## 3.1. Task1, Promise Identification

This is a boolean label (Yes/No) based on whether supporting evidence exists.


### 3.1.1. Task1, Train

In [ ]:
if not torch.cuda.is_available():
    raise RuntimeError("This script requires a GPU to run")

# Create task-specific paths
task1_model_path = os.path.join(task1_output_path, "task1_final_model")
task1_hyperparams_path = os.path.join(task1_output_path, "task1_best_hyperparameters.json")
task1_config_path = os.path.join(task1_output_path, "task1_config.json")  # label_mapping

def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed()

def prepare_model_for_training():
    model = AutoModelForSequenceClassification.from_pretrained(
        "nbroad/ESG-BERT",
        num_labels=2,
        ignore_mismatched_sizes=True
    )

    # Reset the classifier
    model.classifier = torch.nn.Linear(model.config.hidden_size, 2)

    # Freeze all layers except last N transformer layers
    N = 2
    for param in model.bert.parameters():
        param.requires_grad = False
    for param in model.bert.encoder.layer[-N:].parameters():
        param.requires_grad = True
    for param in model.classifier.parameters():
        param.requires_grad = True

    return model

# Load and prepare dataset
print("Loading dataset...")
with open(train_data_path, 'r') as file:
    data = json.load(file)

# Preprocess dataset
print("Processing texts...")
texts = [item['data'] for item in data]
labels = [1 if item["promise_status"] == "Yes" else 0 for item in data]

tokenizer = AutoTokenizer.from_pretrained("nbroad/ESG-BERT")

def tokenize_function(examples):
    return tokenizer(examples, truncation=True, padding="max_length", max_length=512)


# Hyperparameter optimization setup
n_splits = 4
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 12])
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.3)

    fold_losses = []
    for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
        print(f"Trial {trial.number}, Fold {fold + 1}/{n_splits}")

        # Prepare fold data
        train_texts = [texts[i] for i in train_idx]
        val_texts = [texts[i] for i in val_idx]
        train_labels = [labels[i] for i in train_idx]
        val_labels = [labels[i] for i in val_idx]

        train_encodings = tokenize_function(train_texts)
        val_encodings = tokenize_function(val_texts)

        train_dataset = Dataset.from_dict({**train_encodings, "labels": train_labels})
        val_dataset = Dataset.from_dict({**val_encodings, "labels": val_labels})

        model = prepare_model_for_training()
        model.to("cuda")

        trial_dir = os.path.join(task1_output_path, f'trial_{trial.number}_fold_{fold}')

        training_args = TrainingArguments(
            output_dir=trial_dir,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=50,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=10,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            load_best_model_at_end=True,
            no_cuda=False,
            report_to="none",
            seed=42
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        try:
            trainer.train()
            val_output = trainer.evaluate()
            fold_losses.append(val_output["eval_loss"])
        except Exception as e:
            print(f"Error in fold {fold}: {e}")
            return float('inf')
        finally:
            del model, trainer
            torch.cuda.empty_cache()

    return np.mean(fold_losses)

# Run hyperparameter optimization
print("Starting hyperparameter optimization...")
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=7)

# Save best hyperparameters
best_hyperparameters = study.best_params
print("Best Hyperparameters:", best_hyperparameters)
with open(task1_hyperparams_path, "w") as file:
    json.dump(best_hyperparameters, file, indent=2)

# Train final model
print("Training final model...")
train_encodings = tokenize_function(texts)
train_dataset = Dataset.from_dict({**train_encodings, "labels": labels})

final_model = prepare_model_for_training()
final_model.to("cuda")

final_training_args = TrainingArguments(
    output_dir=task1_model_path,
    save_strategy="epoch",
    save_total_limit=2,
    logging_strategy="steps",
    logging_steps=50,
    per_device_train_batch_size=best_hyperparameters["batch_size"],
    num_train_epochs=10,
    learning_rate=best_hyperparameters["learning_rate"],
    weight_decay=best_hyperparameters["weight_decay"],
    no_cuda=False,
    report_to="none",
    seed=42

)

final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

final_trainer.train()

# Save everything needed for prediction
print("Saving model and prediction requirements for task 1...")

# 1. Save the model and tokenizer
final_model.save_pretrained(task1_model_path, save_optimizer_state=False)
tokenizer.save_pretrained(task1_model_path)

# 2. Save promise words and config
config = {
    'label_mapping': {
        '0': 'No',
        '1': 'Yes'
    }
}
with open(task1_config_path, 'w') as f:
    json.dump(config, f, indent=2)

print("Saved the following files for task 1:")
print(f"1. Model and tokenizer at: {task1_model_path}")
print(f"2. Best hyperparameters at: {task1_hyperparams_path}")
print(f"3. Configuration at: {task1_config_path}")
print("\nTask 1 training complete. You can now use these files for prediction on new data.")

Loading dataset...
Processing texts...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

[I 2025-05-18 18:37:26,265] Trial 6 finished with value: 0.4345373436808586 and parameters: {'learning_rate': 2.658616083788978e-05, 'batch_size': 12, 'weight_decay': 0.2900332895916222}. Best is trial 4 with value: 0.43105141818523407.


Best Hyperparameters: {'learning_rate': 2.6771137242145903e-05, 'batch_size': 12, 'weight_decay': 0.1422602954229404}
Training final model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nbroad/ESG-BERT and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([26]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([26, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-1d6c4aaf3015>:172: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(


Step,Training Loss
50,0.516300
100,0.376800
150,0.335100
200,0.276500
250,0.263900
300,0.205400


Saving model and prediction requirements for task 1...
Saved the following files for task 1:
1. Model and tokenizer at: ./promise_task1_processed_Base-Model/task1_final_model
2. Best hyperparameters at: ./promise_task1_processed_Base-Model/task1_best_hyperparameters.json
3. Configuration at: ./promise_task1_processed_Base-Model/task1_config.json

Task 1 training complete. You can now use these files for prediction on new data.


### 3.1.2. Task1, Predict

In [ ]:
base_path = './promise_task1_processed_Base-Model'  # Local path instead of Google Drive
test_data_path = 'english_submission_file_restructured_unlabeled.json'
test_data_T1_labeled_path = 'english_submission_file_restructured_T1-labeled.json'

task1_model_path = os.path.join(base_path, "task1_final_model")
task1_config_path = os.path.join(base_path, "task1_config.json") # contains the label mapping

print("Loading model and tokenizer...")
model = AutoModelForSequenceClassification.from_pretrained(task1_model_path)
tokenizer = AutoTokenizer.from_pretrained(task1_model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load label mapping
with open(task1_config_path, 'r') as f:
    config = json.load(f)
    label_mapping = config['label_mapping']


print("Loading test data...")
with open(test_data_path, 'r') as file:
    test_data = json.load(file)


texts = [item['data'] for item in test_data.values()]

# Tokenize preprocessed texts
def tokenize_function(examples):
    return tokenizer(examples, truncation=True, padding="max_length", max_length=512)

encodings = tokenize_function(texts)
dataset = Dataset.from_dict({
    "input_ids": encodings['input_ids'],
    "attention_mask": encodings['attention_mask']
})

def predict(dataset):
    model.eval()
    trainer = Trainer(model=model, tokenizer=tokenizer)
    raw_pred, _, _ = trainer.predict(dataset)
    predictions = torch.softmax(torch.from_numpy(raw_pred), dim=-1)
    return predictions.argmax(dim=1).numpy()


print("Making predictions...")
predictions = predict(dataset)

# Update the promise_status based on predictions
for idx, pred in enumerate(predictions):
    test_data[str(idx)]['promise_status'] = label_mapping[str(pred)]


print(f"Saving predictions to {test_data_T1_labeled_path}")
with open(test_data_T1_labeled_path, 'w') as file:
    json.dump(test_data, file, indent=4)

print("\nPrediction Statistics:")
pred_distribution = {}
for item in test_data.values():
    status = item['promise_status']
    pred_distribution[status] = pred_distribution.get(status, 0) + 1

for status, count in pred_distribution.items():
    percentage = (count / len(test_data)) * 100
    print(f"{status}: {count} ({percentage:.1f}%)")

Loading model and tokenizer from Drive...
Loading test data...
Making predictions...


<ipython-input-11-7cfea932375d>:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


Saving predictions to english_submission_file_restructured_T1-labeled.json

Prediction Statistics:
Yes: 363 (90.8%)
No: 37 (9.2%)


## 3.2. Task 2, Supporting Evidence

 This is a boolean label (Yes/No) based on whether supporting evidence exists.

### 3.2.1. Task2, Train

In [ ]:
if not torch.cuda.is_available():
    raise RuntimeError("This script requires a GPU to run")

# Create task-specific paths
task2_model_path = os.path.join(task2_output_path, "task2_final_model")
task2_hyperparams_path = os.path.join(task2_output_path, "task2_best_hyperparameters.json")
task2_config_path = os.path.join(task2_output_path, "task2_config.json")  # label_mapping


def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed()


def prepare_model_for_training():
    model = AutoModelForSequenceClassification.from_pretrained(
        "nbroad/ESG-BERT",
        num_labels=2,
        ignore_mismatched_sizes=True
    )

    model.classifier = torch.nn.Linear(model.config.hidden_size, 2)

    # Freeze all layers except last 2 transformer layers and classifier
    N = 2
    for param in model.bert.parameters():
        param.requires_grad = False
    for param in model.bert.encoder.layer[-N:].parameters():
        param.requires_grad = True
    for param in model.classifier.parameters():
        param.requires_grad = True

    return model

# Load dataset
print("Loading dataset...")
with open("PromiseEval_Trainset_English.json", 'r') as file:
    data = json.load(file)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("nbroad/ESG-BERT")

def tokenize_function(examples):
    return tokenizer(examples, truncation=True, padding="max_length", max_length=512)

# Process texts
print("Processing texts...")
texts = [item['data'] for item in data]
labels = [1 if item["evidence_status"] == "Yes" else 0 for item in data]

# Setup k-fold
n_splits = 4
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 12])
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.3)

    fold_losses = []
    for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
        print(f"Trial {trial.number}, Fold {fold + 1}/{n_splits}")

        # Prepare fold data
        train_texts = [texts[i] for i in train_idx]
        val_texts = [texts[i] for i in val_idx]
        train_labels = [labels[i] for i in train_idx]
        val_labels = [labels[i] for i in val_idx]

        train_encodings = tokenize_function(train_texts)
        val_encodings = tokenize_function(val_texts)

        train_dataset = Dataset.from_dict({**train_encodings, "labels": train_labels})
        val_dataset = Dataset.from_dict({**val_encodings, "labels": val_labels})

        model = prepare_model_for_training()
        model.to("cuda")

        trial_dir = os.path.join(task2_output_path, f'trial_{trial.number}_fold_{fold}')

        training_args = TrainingArguments(
            output_dir=trial_dir,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=50,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=10,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            load_best_model_at_end=True,
            no_cuda=False,
            report_to="none",
            seed=42
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        try:
            trainer.train()
            val_output = trainer.evaluate()
            fold_losses.append(val_output["eval_loss"])
        except Exception as e:
            print(f"Error in fold {fold}: {e}")
            return float('inf')
        finally:
            del model, trainer
            torch.cuda.empty_cache()

    return np.mean(fold_losses)

# Run hyperparameter optimization
print("Starting hyperparameter optimization...")
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=7)

# Save best hyperparameters
best_hyperparameters = study.best_params
print("Best Hyperparameters:", best_hyperparameters)
with open(task2_hyperparams_path, 'w') as f:
    json.dump(best_hyperparameters, f, indent=2)

# Train final model
print("Training final model...")
train_encodings = tokenize_function(texts)
train_dataset = Dataset.from_dict({**train_encodings, "labels": labels})

final_model = prepare_model_for_training()
final_model.to("cuda")

final_training_args = TrainingArguments(
    output_dir=task2_model_path,
    save_strategy="epoch",
    save_total_limit=2,
    logging_strategy="steps",
    logging_steps=50,
    per_device_train_batch_size=best_hyperparameters["batch_size"],
    num_train_epochs=10,
    learning_rate=best_hyperparameters["learning_rate"],
    weight_decay=best_hyperparameters["weight_decay"],
    no_cuda=False,
    report_to="none",
    seed=42
)

final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

final_trainer.train()

# Save everything needed for prediction
print("Saving model and prediction requirements for task 2...")

# 1. Save the model and tokenizer
final_model.save_pretrained(task2_model_path, save_optimizer_state=False)
tokenizer.save_pretrained(task2_model_path)

# 2. Save evidence indicators and config
config = {
    'label_mapping': {
        '0': 'No',
        '1': 'Yes'
    }
}

with open(task2_config_path, 'w') as f:
    json.dump(config, f, indent=2)

print("Saved the following files for task 2:")
print(f"1. Model and tokenizer at: {task2_model_path}")
print(f"2. Configuration at: {task2_config_path}")
print(f"3. Best hyperparameters at: {task2_hyperparams_path}")
print("\nTask 2 training complete. You can now use these files for prediction on new data.")

Loading dataset...


[I 2025-05-19 00:38:26,679] A new study created in memory with name: no-name-49e7d57f-f4ee-4b00-bca9-e8333fd20a72


Processing texts...
Starting hyperparameter optimization...
Trial 0, Fold 1/4


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nbroad/ESG-BERT and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([26]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([26, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-9c09d2011753>:106: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.683400,0.572438
2,0.560000,0.557496
3,0.505900,0.552609
4,0.474100,0.533150
5,0.438700,0.547783
6,0.398300,0.546339


...

[I 2025-05-19 00:56:21,937] Trial 6 finished with value: 0.5836943835020065 and parameters: {'learning_rate': 2.658616083788978e-05, 'batch_size': 12, 'weight_decay': 0.2900332895916222}. Best is trial 4 with value: 0.572898805141449.


Best Hyperparameters: {'learning_rate': 2.6771137242145903e-05, 'batch_size': 12, 'weight_decay': 0.1422602954229404}
Training final model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nbroad/ESG-BERT and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([26]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([26, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-9c09d2011753>:162: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(


Step,Training Loss
50,0.629000
100,0.509800
150,0.447100
200,0.389900
250,0.351600
300,0.317700


Saving model and prediction requirements for task 2...
Saved the following files for task 2:
1. Model and tokenizer at: ./promise_task2_processed_Base-Model/task2_final_model
2. Configuration at: ./promise_task2_processed_Base-Model/task2_config.json
3. Best hyperparameters at: ./promise_task2_processed_Base-Model/task2_best_hyperparameters.json

Task 2 training complete. You can now use these files for prediction on new data.


### 3.2.2. Task2, Predict

In [ ]:
test_data_T1_labeled_path = 'english_submission_file_restructured_T1-labeled.json' # annotted from the previous step
test_data_T1_T2_labeled_path = 'english_submission_file_restructured_T1_T2-labeled.json'

task2_model_path = os.path.join(task2_output_path, "task2_final_model")
task2_config_path = os.path.join(task2_output_path, "task2_config.json")

print("Loading model and tokenizer...")
model = AutoModelForSequenceClassification.from_pretrained(task2_model_path)
tokenizer = AutoTokenizer.from_pretrained(task2_model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load label_mapping
with open(task2_config_path, 'r') as f:
    config = json.load(f)
    label_mapping = config['label_mapping']

print("Loading test data...")
with open(test_data_T1_labeled_path, 'r') as file:
    test_data = json.load(file)

# Extract texts for prediction
texts = [item['data'] for item in test_data.values()]

# Tokenize preprocessed texts
def tokenize_function(examples):
    return tokenizer(examples, truncation=True, padding="max_length", max_length=512)

encodings = tokenize_function(texts)
dataset = Dataset.from_dict({
    "input_ids": encodings['input_ids'],
    "attention_mask": encodings['attention_mask']
})

# Prediction function
def predict(dataset):
    model.eval()
    trainer = Trainer(model=model, tokenizer=tokenizer)
    raw_pred, _, _ = trainer.predict(dataset)
    predictions = torch.softmax(torch.from_numpy(raw_pred), dim=-1)
    return predictions.argmax(dim=1).numpy()

# Make predictions
print("Making predictions...")
predictions = predict(dataset)

# Update the evidence_status based on predictions
for idx, pred in enumerate(predictions):
    test_data[str(idx)]['evidence_status'] = label_mapping[str(pred)]

print(f"Saving predictions to {test_data_T1_T2_labeled_path}")
with open(test_data_T1_T2_labeled_path, 'w') as file:
    json.dump(test_data, file, indent=4)

print("\nPrediction Statistics:")
pred_distribution = {}
for item in test_data.values():
    status = item['evidence_status']
    pred_distribution[status] = pred_distribution.get(status, 0) + 1

for status, count in pred_distribution.items():
    percentage = (count / len(test_data)) * 100
    print(f"{status}: {count} ({percentage:.1f}%)")

Loading model and tokenizer...
Loading test data...
Making predictions...


<ipython-input-15-0455f895b6b1>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


Saving predictions to english_submission_file_restructured_T1_T2-labeled.json

Prediction Statistics:
No: 168 (42.0%)
Yes: 232 (58.0%)


below is the one with zip. to be waitied

In [ ]:
base_path = './promise_task2_processed_Base-Model'  # Local path instead of Google Drive
test_data_path = 'english_submission_file_restructured_T1-labeled.json'
test_data_T2_labeled_path = 'english_submission_file_restructured_T1_T2-labeled.json'

task2_model_path = os.path.join(base_path, "task2_final_model")
task2_config_path = os.path.join(base_path, "task2_config.json") # contains the label mapping

print("Loading model and tokenizer...")
model = AutoModelForSequenceClassification.from_pretrained(task2_model_path)
tokenizer = AutoTokenizer.from_pretrained(task2_model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Loading test data...")
with open(test_data_path, 'r') as file:
    test_data = json.load(file)


print("Loading label mapping...")
with open(task2_config_path, 'r') as f:
    config = json.load(f)
    label_mapping = config['label_mapping']

print("Processing texts...")
texts = []
indices = []
for idx in sorted(test_data.keys(), key=int):
    item = test_data[idx]
    input_text = item['data']
    texts.append(input_text)
    indices.append(idx)

# Tokenize all texts
encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=512)
dataset = Dataset.from_dict({
    "input_ids": encodings['input_ids'],
    "attention_mask": encodings['attention_mask']
})

# Make predictions
print("Making predictions...")
model.eval()
trainer = Trainer(model=model, tokenizer=tokenizer)
predictions = trainer.predict(dataset)
predicted_labels = predictions.predictions.argmax(-1)

# Update test data with predictions
for idx, pred in zip(indices, predicted_labels):
    test_data[idx]['evidence_status'] = label_mapping[str(pred)]

# Save updated test data
output_path = os.path.join(test_data_T2_labeled_path)
print(f"Saving predictions to {output_path}")
with open(output_path, 'w') as f:
    json.dump(test_data, f, indent=2)

# Print prediction statistics
print("\nPrediction Statistics:")
pred_distribution = {}
for idx in test_data:
    status = test_data[idx]['evidence_status']
    pred_distribution[status] = pred_distribution.get(status, 0) + 1

for status, count in pred_distribution.items():
    percentage = (count / len(test_data)) * 100
    print(f"{status}: {count} ({percentage:.1f}%)")

Loading model and tokenizer...
Loading test data...
Loading label mapping...
Processing texts...
Making predictions...


<ipython-input-20-b4453c3b1d35>:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


Saving predictions to english_submission_file_restructured_T1_T2-labeled.json

Prediction Statistics:
No: 168 (42.0%)
Yes: 232 (58.0%)


## 3.3. Task 3, Clarity of the Promise-Evidence Pair

We designed three labels (Clear/Not Clear/Misleading) for this task, which should depend on the clarity of the given evidence in relation to the promise.

### 3.3.1. Task3,Train

In [ ]:

# Create task-specific paths
task3_model_path = os.path.join(task3_output_path, "task3_final_model")
task3_hyperparams_path = os.path.join(task3_output_path, "task3_best_hyperparameters.json")
task3_config_path = os.path.join(task3_output_path, "task3_config.json")  # label_mapping

if not torch.cuda.is_available():
    raise RuntimeError("This script requires a GPU to run")

def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed()


def prepare_model_for_training():
    model = AutoModelForSequenceClassification.from_pretrained(
        "nbroad/ESG-BERT",
        num_labels=4,  # Clear, Not Clear, Misleading, N/A
        ignore_mismatched_sizes=True
    )

    # Freeze all layers except last N transformer layers
    N = 2
    for param in model.bert.parameters():
        param.requires_grad = False
    for param in model.bert.encoder.layer[-N:].parameters():
        param.requires_grad = True
    for param in model.classifier.parameters():
        param.requires_grad = True

    return model

# Load dataset
print("Loading dataset...")
with open("PromiseEval_Trainset_English.json", 'r') as file:
    data = json.load(file)

clarity_to_id = {
    "Clear": 0,
    "Not Clear": 1,
    "Misleading": 2,
    "N/A": 3
}

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("nbroad/ESG-BERT")

def tokenize_function(examples):
    return tokenizer(examples, truncation=True, padding="max_length", max_length=512)

# Preprocess dataset
print("Processing texts...")
texts = [item['data']for item in data]
labels = [clarity_to_id[item["evidence_quality"]] for item in data]


# Setup cross-validation
n_splits = 4
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)


def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 12])
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.3)

    fold_losses = []
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
        print(f"Trial {trial.number}, Fold {fold + 1}/{n_splits}")

        # Prepare fold data
        train_texts = [texts[i] for i in train_idx]
        val_texts = [texts[i] for i in val_idx]
        train_labels = [labels[i] for i in train_idx]
        val_labels = [labels[i] for i in val_idx]

        train_encodings = tokenize_function(train_texts)
        val_encodings = tokenize_function(val_texts)

        train_dataset = Dataset.from_dict({**train_encodings, "labels": train_labels})
        val_dataset = Dataset.from_dict({**val_encodings, "labels": val_labels})

        model = prepare_model_for_training()
        model.to("cuda")

        trial_dir = os.path.join(task3_output_path, f'trial_{trial.number}_fold_{fold}')

        training_args = TrainingArguments(
            output_dir=trial_dir,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=50,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=10,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            load_best_model_at_end=True,

            no_cuda=False,
            report_to="none",
            seed=42
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        try:
            trainer.train()
            eval_results = trainer.evaluate()
            fold_losses.append(eval_results["eval_loss"])
        except Exception as e:
            print(f"Error in fold {fold}: {e}")
            return float('inf')
        finally:
            del model, trainer
            torch.cuda.empty_cache()

    return np.mean(fold_losses)

# Run hyperparameter optimization
print("Starting hyperparameter optimization...")
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=7)

# Save best hyperparameters
best_hyperparameters = study.best_params
print("Best Hyperparameters:", best_hyperparameters)
with open(task3_hyperparams_path, "w") as file:
    json.dump(best_hyperparameters, file)

# Train final model
print("Training final model...")
train_encodings = tokenize_function(texts)
train_dataset = Dataset.from_dict({**train_encodings, "labels": labels})

final_model = prepare_model_for_training()
final_model.to("cuda")

final_training_args = TrainingArguments(
    output_dir=task3_model_path,
    save_strategy="epoch",
    save_total_limit=2,
    logging_strategy="steps",
    logging_steps=50,
    per_device_train_batch_size=best_hyperparameters["batch_size"],
    num_train_epochs=10,
    learning_rate=best_hyperparameters["learning_rate"],
    weight_decay=best_hyperparameters["weight_decay"],
    no_cuda=False,
    report_to="none",
    seed=42
)

final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
)

final_trainer.train()

# Save final model
print("Saving final model...")
final_model.save_pretrained(task3_model_path, save_optimizer_state=False)
tokenizer.save_pretrained(task3_model_path)

# Save class mapping for future reference
class_mapping = {v: k for k, v in clarity_to_id.items()}
config = {
    'label_mapping': class_mapping,
}

with open(task3_config_path, "w") as f:
    json.dump(config, f) # saving the lable mapping

print("Saved the following files for task 3:")
print(f"1. Model and tokenizer at: {task3_model_path}")
print(f"2. Configuration at: {task3_config_path}")
print(f"3. Best hyperparameters at: {task3_hyperparams_path}")
print("\nTask 3 training complete. You can now use these files for prediction on new data.")


Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

[I 2025-05-19 16:41:45,208] Trial 6 finished with value: 1.0026298016309738 and parameters: {'learning_rate': 2.658616083788978e-05, 'batch_size': 12, 'weight_decay': 0.2900332895916222}. Best is trial 6 with value: 1.0026298016309738.


Best Hyperparameters: {'learning_rate': 2.658616083788978e-05, 'batch_size': 12, 'weight_decay': 0.2900332895916222}
Training final model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nbroad/ESG-BERT and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([26]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([26, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-83741cff53d8>:170: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(


Step,Training Loss
50,1.137600
100,0.939600
150,0.857500
200,0.764900
250,0.725700
300,0.672000


Saving final model...
Saved the following files for task 3:
1. Model and tokenizer at: ./promise_task3_processed_Base-Model/task3_final_model
2. Configuration at: ./promise_task3_processed_Base-Model/task3_config.json
3. Best hyperparameters at: ./promise_task3_processed_Base-Model/task3_best_hyperparameters.json

Task 3 training complete. You can now use these files for prediction on new data.


### 3.3.2. Task3, Predict

In [ ]:
test_data_T1_T2_labeled_path = 'english_submission_file_restructured_T1_T2-labeled.json'
test_data_T1_T2_T3_labeled_path = 'english_submission_file_restructured_T1_T2_T3-labeled.json'

task3_model_path = os.path.join(task3_output_path, "task3_final_model")
task3_config_path = os.path.join(task3_output_path, "task3_config.json")

print("Loading model and tokenizer...")
model = AutoModelForSequenceClassification.from_pretrained(task3_model_path)
tokenizer = AutoTokenizer.from_pretrained(task3_model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load label_mapping
with open(task3_config_path, 'r') as f:
    config = json.load(f)
    label_mapping = config['label_mapping']

print("Loading test data...")
with open(test_data_T1_T2_labeled_path, 'r') as file:
    test_data = json.load(file)

# Extract texts for prediction
texts = [item['data'] for item in test_data.values()]

# Tokenize preprocessed texts
def tokenize_function(examples):
    return tokenizer(examples, truncation=True, padding="max_length", max_length=512)

encodings = tokenize_function(texts)
dataset = Dataset.from_dict({
    "input_ids": encodings['input_ids'],
    "attention_mask": encodings['attention_mask']
})

# Prediction function
def predict(dataset):
    model.eval()
    trainer = Trainer(model=model, tokenizer=tokenizer)
    raw_pred, _, _ = trainer.predict(dataset)
    predictions = torch.softmax(torch.from_numpy(raw_pred), dim=-1)
    return predictions.argmax(dim=1).numpy()

# Make predictions
print("Making predictions...")
predictions = predict(dataset)

# Update the evidence_quality based on predictions
for idx, pred in enumerate(predictions):
    test_data[str(idx)]['evidence_quality'] = label_mapping[str(pred)]

print(f"Saving predictions to {test_data_T1_T2_T3_labeled_path}")
with open(test_data_T1_T2_T3_labeled_path, 'w') as file:
    json.dump(test_data, file, indent=4)

print("\nPrediction Statistics:")
pred_distribution = {}
for item in test_data.values():
    status = item['evidence_quality']
    pred_distribution[status] = pred_distribution.get(status, 0) + 1

for status, count in pred_distribution.items():
    percentage = (count / len(test_data)) * 100
    print(f"{status}: {count} ({percentage:.1f}%)")

Loading model and tokenizer...
Loading test data...
Making predictions...


<ipython-input-8-d314fddb3d80>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


Saving predictions to english_submission_file_restructured_T1_T2_T3-labeled.json

Prediction Statistics:
N/A: 216 (54.0%)
Clear: 171 (42.8%)
Not Clear: 13 (3.2%)


## 3.4. Task4, Timing for Verification

Following the MSCI guidelines, we set timing labels (within 2 years/2-5 years/longer than 5 years/other) to indicate when readers/investors should return to verify the promise. Here, "other" denotes the promise has already been verified or doesn't have a specific timing to verify it.

### 3.4.1. Task4, Train



In [ ]:
# Create task-specific paths
task4_model_path = os.path.join(task4_output_path, "task4_final_model")
task4_hyperparams_path = os.path.join(task4_output_path, "task4_best_hyperparameters.json")
task4_config_path = os.path.join(task4_output_path, "task4_config.json")  # label_mapping

if not torch.cuda.is_available():
    raise RuntimeError("This script requires a GPU to run")

def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed()

def prepare_model_for_training():
    model = AutoModelForSequenceClassification.from_pretrained(
        "nbroad/ESG-BERT",
        num_labels=5,
        ignore_mismatched_sizes=True
    )

    model.classifier = torch.nn.Linear(model.config.hidden_size, 5)

    # Freeze all layers except last N transformer layers
    N = 2
    for param in model.bert.parameters():
        param.requires_grad = False
    for param in model.bert.encoder.layer[-N:].parameters():
        param.requires_grad = True
    for param in model.classifier.parameters():
        param.requires_grad = True

    return model

print("Loading dataset...")
with open("PromiseEval_Trainset_English.json", 'r') as file:
    data = json.load(file)

timeline_to_id = {
    "Less than 2 years": 0,
    "2 to 5 years": 1,
    "More than 5 years": 2,
    "Already": 3,
    "N/A": 4
}

tokenizer = AutoTokenizer.from_pretrained("nbroad/ESG-BERT")

def tokenize_function(examples):
    return tokenizer(examples, truncation=True, padding="max_length", max_length=512)

# Enhanced text preprocessing
print("Processing texts...")
texts = [item['data'] for item in data]
labels = [timeline_to_id[item["verification_timeline"].strip()] for item in data]

# Setup k-fold
n_splits = 4
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 12])
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.3)

    fold_losses = []
    for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
        print(f"Trial {trial.number}, Fold {fold + 1}/{n_splits}")

        # Prepare fold data
        train_texts = [texts[i] for i in train_idx]
        val_texts = [texts[i] for i in val_idx]
        train_labels = [labels[i] for i in train_idx]
        val_labels = [labels[i] for i in val_idx]

        train_encodings = tokenize_function(train_texts)
        val_encodings = tokenize_function(val_texts)

        train_dataset = Dataset.from_dict({**train_encodings, "labels": train_labels})
        val_dataset = Dataset.from_dict({**val_encodings, "labels": val_labels})

        model = prepare_model_for_training()
        model.to("cuda")

        trial_dir = os.path.join(task4_output_path, f'trial_{trial.number}_fold_{fold}')

        training_args = TrainingArguments(
            output_dir=trial_dir,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=50,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=10,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            load_best_model_at_end=True,
            no_cuda=False,
            report_to="none",
            seed=42

        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        try:
            trainer.train()
            val_output = trainer.evaluate()
            fold_losses.append(val_output["eval_loss"])
        except Exception as e:
            print(f"Error in fold {fold}: {e}")
            return float('inf')
        finally:
            del model, trainer
            torch.cuda.empty_cache()

    return np.mean(fold_losses)

# Run hyperparameter optimization
print("Starting hyperparameter optimization...")
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=7)

# Save best hyperparameters
best_hyperparameters = study.best_params
print("Best Hyperparameters:", best_hyperparameters)

# Train final model
print("Training final model...")
train_encodings = tokenize_function(texts)
train_dataset = Dataset.from_dict({**train_encodings, "labels": labels})

final_model = prepare_model_for_training()
final_model.to("cuda")


final_training_args = TrainingArguments(
    output_dir=task4_model_path,
    save_strategy="epoch",
    save_total_limit=2,
    logging_strategy="steps",
    logging_steps=50,
    per_device_train_batch_size=best_hyperparameters["batch_size"],
    num_train_epochs=10,
    learning_rate=best_hyperparameters["learning_rate"],
    weight_decay=best_hyperparameters["weight_decay"],
    no_cuda=False,
    report_to="none",
    seed=42

)

final_trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

final_trainer.train()

# Save everything needed for prediction
print("Saving model and prediction requirements for task 4...")

# 1. Save the model and tokenizer
final_model.save_pretrained(task4_model_path, save_optimizer_state=False)
tokenizer.save_pretrained(task4_model_path)

# 3. Save label mapping
label_mapping = {
    'label_mapping': {v: k for k, v in timeline_to_id.items()},
}

with open(task4_config_path, 'w') as f:
    json.dump(label_mapping, f, indent=2)

# 4. Save best hyperparameters
with open(task4_hyperparams_path, 'w') as f:
    json.dump(best_hyperparameters, f, indent=2)

print("Saved the following files for task 4 prediction:")
print(f"1. Model and tokenizer at: {task4_model_path}")
print(f"2. Label mapping at: {task4_config_path}")
print(f"3. Best hyperparameters at: {task4_hyperparams_path}")
print("\nTask 4 training complete. You can now use these files for prediction on new data.")

Loading dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

[I 2025-05-19 19:27:15,222] Trial 6 finished with value: 1.3426111340522766 and parameters: {'learning_rate': 2.658616083788978e-05, 'batch_size': 12, 'weight_decay': 0.2900332895916222}. Best is trial 5 with value: 1.3379151225090027.


Best Hyperparameters: {'learning_rate': 3.538461259525519e-05, 'batch_size': 12, 'weight_decay': 0.02347061968879934}
Training final model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nbroad/ESG-BERT and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([26]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([26, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-407b99f788e7>:166: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(


Step,Training Loss
50,1.389900
100,1.176600
150,1.065600
200,1.001700
250,0.887500
300,0.867900


Saving model and prediction requirements for task 4...
Saved the following files for task 4 prediction:
1. Model and tokenizer at: ./promise_task4_processed_Base-Model/task4_final_model
2. Label mapping at: ./promise_task4_processed_Base-Model/task4_config.json
3. Best hyperparameters at: ./promise_task4_processed_Base-Model/task4_best_hyperparameters.json

Task 4 training complete. You can now use these files for prediction on new data.


### 3.4.2. Task4, Predict

In [ ]:
test_data_T1_T2_T3_labeled_path = 'english_submission_file_restructured_T1_T2_T3-labeled.json' # input
test_data_T1_T2_T3_T4_labeled_path = 'english_submission_file_restructured_T1_T2_T3_T4-labeled.json' # output

task4_model_path = os.path.join(task4_output_path, "task4_final_model")
task4_config_path = os.path.join(task4_output_path, "task4_config.json")

print("Loading model and tokenizer...")
model = AutoModelForSequenceClassification.from_pretrained(task4_model_path)
tokenizer = AutoTokenizer.from_pretrained(task4_model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load label_mapping
with open(task4_config_path, 'r') as f:
    config = json.load(f)
    label_mapping = config['label_mapping']

print("Loading test data...")
with open(test_data_T1_T2_T3_labeled_path, 'r') as file:
    test_data = json.load(file)

texts = [item['data'] for item in test_data.values()]

# Tokenize preprocessed texts
def tokenize_function(examples):
    return tokenizer(examples, truncation=True, padding="max_length", max_length=512)

encodings = tokenize_function(texts)
dataset = Dataset.from_dict({
    "input_ids": encodings['input_ids'],
    "attention_mask": encodings['attention_mask']
})

# Prediction function
def predict(dataset):
    model.eval()
    trainer = Trainer(model=model, tokenizer=tokenizer)
    raw_pred, _, _ = trainer.predict(dataset)
    predictions = torch.softmax(torch.from_numpy(raw_pred), dim=-1)
    return predictions.argmax(dim=1).numpy()

# Make predictions
print("Making predictions...")
predictions = predict(dataset)

# Update the verification_timeline based on predictions
for idx, pred in enumerate(predictions):
    test_data[str(idx)]['verification_timeline'] = label_mapping[str(pred)]

print(f"Saving predictions to {test_data_T1_T2_T3_T4_labeled_path}")
with open(test_data_T1_T2_T3_T4_labeled_path, 'w') as file:
    json.dump(test_data, file, indent=4)

print("\nPrediction Statistics:")
pred_distribution = {}
for item in test_data.values():
    timeline = item['verification_timeline']
    pred_distribution[timeline] = pred_distribution.get(timeline, 0) + 1

for timeline, count in pred_distribution.items():
    percentage = (count / len(test_data)) * 100
    print(f"{timeline}: {count} ({percentage:.1f}%)")

Loading model and tokenizer...
Loading test data...
Making predictions...


<ipython-input-8-e7c36f0f3c29>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, tokenizer=tokenizer)


Saving predictions to english_submission_file_restructured_T1_T2_T3_T4-labeled.json

Prediction Statistics:
N/A: 66 (16.5%)
2 to 5 years: 90 (22.5%)
Already: 190 (47.5%)
Less than 2 years: 14 (3.5%)
More than 5 years: 40 (10.0%)


# 4. Convert to Parquet for Kaggle Submission










As required by the SemEval-2025 PromiseEval competition on Kaggle, submissions must be in the Parquet file format. The code below converts our final prediction JSON file containing all four tasks' predictions into the required Parquet format for submission.

In [ ]:
import pandas as pd
import json

with open("english_submission_file_restructured_T1_T2_T3_T4-labeled.json", 'r') as file:
    test_data = json.load(file)

df = pd.DataFrame.from_dict(test_data, orient='index')
df.to_parquet("test_data_base-model_T1_T2_T3_T4_submit.parquet")

print("Conversion complete! You may now submit the parquet file.")

Conversion complete!
